In [58]:
import pandas as pd

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec, LdaMulticore
from gensim.utils import simple_preprocess

from gensim.corpora import Dictionary
import pyLDAvis
from pyLDAvis import gensim 

In [4]:
df = pd.read_csv('data/job_ofer.csv')
df.shape

(36109, 8)

In [5]:
df.head()

,title,company_name,address,description,seniority_level,employment_type,job_function,industries
0,Machine Learning Engineer,Intellipro Group Inc,"Palo Alto, CA, US","['About The Company', ""W*** is reshaping the f...",Entry level,Full-time,Engineering,Information Technology and Services
1,Deep Learning Applied Researcher - Chicago,Ethosia,"Chicago, IL, US","['תיאור המשרה', 'Deep learning for Computer Vi...",Associate,Full-time,Other,Information Technology and Services
2,Machine Learning Engineer,Motorola Solutions,"Chicago, IL, US","['Company Overview', 'At Motorola Solutions, w...",Entry level,Full-time,Engineering,Information Technology and Services
3,Machine Learning / Data Scientist,Proprius LLC,"San Francisco, CA, US",['Our client is a digital invention agency foc...,Entry level,Full-time,Engineering,Information Technology and Services
4,Cloud Architect,TCS,"Framingham, Massachusetts, United States","['Technical/Functional Skills', ' ', 'Good to ...",Mid-Senior level,Full-time,Engineering,Information Technology and Services


# Word2Vec

In [14]:
title_corpus = df.title.map(simple_preprocess)

In [15]:
model = Word2Vec(title_corpus, size=100, window = 2, min_count=1)

In [16]:
model.most_similar('machine')

/home/sendivogius/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('deep', 0.9518992900848389),
 ('captivate', 0.8853990435600281),
 ('edge', 0.8529528379440308),
 ('big', 0.8455701470375061),
 ('inference', 0.8285239934921265),
 ('computer', 0.8226865530014038),
 ('predictive', 0.8184211254119873),
 ('natural', 0.8160594701766968),
 ('scientist', 0.810634970664978),
 ('gir', 0.8085177540779114)]

## Title + phrases

In [45]:
title_bigram = Phraser(Phrases(title_corpus, min_count=1, threshold=1))

In [34]:
def prepare_corpus(corpus, bigram):
    for sent in corpus:
        yield  bigram[sent] + sent

In [46]:
ext_corpus_title = list(prepare_corpus(title_corpus, title_bigram))

In [47]:
title_model = Word2Vec(ext_corpus_title, size=100, window = 2, min_count=1)

In [48]:
title_model.most_similar('nlp')

/home/sendivogius/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('alpharetta', 0.9900477528572083),
 ('artificial_intelligence', 0.9888246655464172),
 ('ml', 0.986015796661377),
 ('computer_vision', 0.982696533203125),
 ('applied_research', 0.9824648499488831),
 ('computer', 0.9819604754447937),
 ('big_data', 0.9808465242385864),
 ('deep', 0.979270339012146),
 ('deep_learning', 0.9779534339904785),
 ('scientist_nlp', 0.9775915145874023)]

## Descriptions + phrases

In [41]:
desc_corpus = df.description.map(simple_preprocess)
desc_bigram = Phraser(Phrases(desc_corpus, min_count=1, threshold=1))

In [42]:
ext_corpus_desc = list(prepare_corpus(desc_corpus, desc_bigram))

In [43]:
desc_model = Word2Vec(ext_corpus_desc, size=100, window=2, min_count=1)

# LDA na tytułach

In [53]:
dictionary = Dictionary(ext_corpus_title)

CPU times: user 704 ms, sys: 3.97 ms, total: 708 ms
Wall time: 709 ms


In [54]:
bow_corpus = [dictionary.doc2bow(sent) for sent in ext_corpus_title]

In [59]:
lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=50, workers=4)

In [60]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

/home/sendivogius/miniconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
